In [7]:
import nibabel as nb
import numpy as np
import Functional_Fusion.atlas_map as am
import Functional_Fusion.dataset as ds
import matplotlib.pyplot as plt
import SUITPy as suit 
import nitools as nt 
import Functional_Fusion.plot as plot
from matplotlib.colors import ListedColormap
import pandas as pd
import seaborn as sb
from scipy import stats 
import copy 

base_dir = '/Volumes/diedrichsen_data$/data/FunctionalFusion' 
atlas_dir = base_dir + '/Atlases/tpl-MNI152NLin2009cSymC'
wk_dir = '/Volumes/diedrichsen_data$/data/Cerebellum/Pontine7T/atlases/dentate'


In [35]:
#these are KxP matrices for each atlas, 32 parcels 

pmap_all = np.load(f"{wk_dir}/Prob_dentate.npy")
pmap_mdtb = np.load(f"{wk_dir}/Prob_dentate_mdtb(ses1).npy") 
pmap_mdtb_ses2 = np.load(f"{wk_dir}/Prob_dentate_mdtb(ses2).npy") 
pmap_pontine = np.load(f"{wk_dir}/Prob_dentate_pontine.npy")
pmap_lang = np.load(f"{wk_dir}/Prob_dentate_language.npy")

print(pmap_all.shape)



(32, 3934)


In [40]:
#repeating pearson correlation with randomization test

n_randomizations = 100

pmap_flat = pmap_all.flatten()
pmap_mdtb_flat = pmap_mdtb.flatten()
pmap_mdtb_ses2_flat = pmap_mdtb_ses2.flatten()
pmap_pontine_flat = pmap_pontine.flatten()
pmap_lang_flat = pmap_lang.flatten()

pairs = [
    ("pmap vs mdtb", pmap_flat, pmap_mdtb_flat),
    ("pmap vs pontine", pmap_flat, pmap_pontine_flat),
    ("pmap vs lang", pmap_flat, pmap_lang_flat),
    ("mdtb vs pontine", pmap_mdtb_flat, pmap_pontine_flat),
    ("mdtb vs lang", pmap_mdtb_flat, pmap_lang_flat),
    ("pontine vs lang", pmap_pontine_flat, pmap_lang_flat),
    ("mdtb ses1 vs ses2", pmap_mdtb_flat, pmap_mdtb_ses2_flat),
    ("pontine vs mdtb_ses2", pmap_pontine_flat, pmap_mdtb_ses2_flat),
    ("lang vs mdtb_ses2", pmap_lang_flat, pmap_mdtb_ses2_flat),
]

# Function for shuffling all matrix rows randomly 
def shuffle_rows(matrix):
    shuffled = matrix.copy()
    np.random.shuffle(shuffled)  
    return shuffled

# Store original results
original_results = {}
for name, arr1, arr2 in pairs:
    corr, p_value = stats.pearsonr(arr1, arr2)
    original_results[name] = (corr, p_value)

# Perform randomization test
randomized_results = {name: [] for name, _, _ in pairs}

for i in range(n_randomizations):
    #print(f"Randomization {i+1}/{n_randomizations}")

    # Shuffle rows 
    pmap_mdtb_shuffled = shuffle_rows(pmap_mdtb)
    pmap_mdtb_ses2_shuffled = shuffle_rows(pmap_mdtb_ses2)
    pmap_pontine_shuffled = shuffle_rows(pmap_pontine)
    pmap_lang_shuffled = shuffle_rows(pmap_lang)

    pmap_mdtb_flat_shuffled = pmap_mdtb_shuffled.flatten()
    pmap_pontine_flat_shuffled = pmap_pontine_shuffled.flatten()
    pmap_lang_flat_shuffled = pmap_lang_shuffled.flatten()
    pmap_mdtb_ses2_flat_shuffled = pmap_mdtb_ses2_shuffled.flatten()

    # Compute correlations with shuffled data
    shuffled_pairs = [
        ("pmap vs mdtb", pmap_flat, pmap_mdtb_flat_shuffled),
        ("pmap vs pontine", pmap_flat, pmap_pontine_flat_shuffled),
        ("pmap vs lang", pmap_flat, pmap_lang_flat_shuffled),

        ("mdtb vs pontine", pmap_mdtb_flat_shuffled, pmap_pontine_flat_shuffled),
        ("mdtb vs lang", pmap_mdtb_flat_shuffled, pmap_lang_flat_shuffled),
        ("pontine vs lang", pmap_pontine_flat_shuffled, pmap_lang_flat_shuffled),
        ("mdtb ses1 vs ses2", pmap_mdtb_flat_shuffled, pmap_mdtb_ses2_flat_shuffled),
        ("pontine vs mdtb_ses2", pmap_pontine_flat_shuffled, pmap_mdtb_ses2_flat_shuffled),
        ("lang vs mdtb_ses2", pmap_lang_flat_shuffled, pmap_mdtb_ses2_flat_shuffled),
    ]

    for name, arr1, arr2 in shuffled_pairs:
        corr, p_value = stats.pearsonr(arr1, arr2)
        randomized_results[name].append(corr)  

print("\n=== Original vs. Randomized Correlation Results ===")
for name in original_results.keys():
    orig_corr, orig_p = original_results[name]
    rand_corr_mean = np.mean(randomized_results[name]) #computing mean tells us what the expected correlation would be if the data was random (each random test reveals a possible correlation value on a distribution)
    rand_corr_std = np.std(randomized_results[name])

    t_stat, t_p_value = stats.ttest_1samp(randomized_results[name], orig_corr)
    
    print(f"{name}:")
    print(f"  Original Corr: {orig_corr:.4f}")
    print(f"  Randomized Corr (mean ± std): {rand_corr_mean:.4f} ± {rand_corr_std:.4f}")
    print(f"  T-statistic: {t_stat:.4f}, p-value: {t_p_value:.4f}")  # Added t-test results
    print("-" * 50)



=== Original vs. Randomized Correlation Results ===
pmap vs mdtb:
  Original Corr: 0.4419
  Randomized Corr (mean ± std): -0.0013 ± 0.0418
  T-statistic: -105.4352, p-value: 0.0000
--------------------------------------------------
pmap vs pontine:
  Original Corr: 0.4094
  Randomized Corr (mean ± std): -0.0006 ± 0.0250
  T-statistic: -163.4918, p-value: 0.0000
--------------------------------------------------
pmap vs lang:
  Original Corr: 0.3437
  Randomized Corr (mean ± std): -0.0009 ± 0.0334
  T-statistic: -102.5328, p-value: 0.0000
--------------------------------------------------
mdtb vs pontine:
  Original Corr: 0.0034
  Randomized Corr (mean ± std): 0.0024 ± 0.0184
  T-statistic: -0.5438, p-value: 0.5878
--------------------------------------------------
mdtb vs lang:
  Original Corr: 0.0147
  Randomized Corr (mean ± std): -0.0015 ± 0.0281
  T-statistic: -5.7111, p-value: 0.0000
--------------------------------------------------
pontine vs lang:
  Original Corr: 0.0056
  Ran

In [22]:
#KxP matrices for each atlas, 5 parcels 

pmap_all_5parcels = np.load(f"{wk_dir}/Prob_dentate_5parcels.npy")
pmap_mdtb_5parcels = np.load(f"{wk_dir}/Prob_dentate_mdtb(ses1)_5parcels.npy")  
pmap_pontine_5parcels = np.load(f"{wk_dir}/Prob_dentate_pontine_5parcels.npy")
pmap_lang_5parcels = np.load(f"{wk_dir}/Prob_dentate_language_5parcels.npy")

print(pmap_all_5parcels.shape)



(4, 3934)


In [27]:
#repeating pearson correlation with randomization test 5 parcels 

n_randomizations = 100

pmap_flat = pmap_all_5parcels.flatten()
pmap_mdtb_flat = pmap_mdtb_5parcels.flatten()
pmap_pontine_flat = pmap_pontine_5parcels.flatten()
pmap_lang_flat = pmap_lang_5parcels.flatten()

pairs = [
    ("pmap vs mdtb", pmap_flat, pmap_mdtb_flat),
    ("pmap vs pontine", pmap_flat, pmap_pontine_flat),
    ("pmap vs lang", pmap_flat, pmap_lang_flat),
    ("mdtb vs pontine", pmap_mdtb_flat, pmap_pontine_flat),
    ("mdtb vs lang", pmap_mdtb_flat, pmap_lang_flat),
    ("pontine vs lang", pmap_pontine_flat, pmap_lang_flat),
]

# Function for shuffling matrix rows 
def shuffle_rows(matrix):
    shuffled = matrix.copy()
    np.random.shuffle(shuffled)  
    return shuffled

# Store original results
original_results = {}
for name, arr1, arr2 in pairs:
    corr, p_value = stats.pearsonr(arr1, arr2)
    original_results[name] = (corr, p_value)

# Perform randomization test
randomized_results = {name: [] for name, _, _ in pairs}

for i in range(n_randomizations):
    #print(f"Randomization {i+1}/{n_randomizations}")

    # Shuffle rows 
    pmap_mdtb_shuffled = shuffle_rows(pmap_mdtb_5parcels)
    pmap_pontine_shuffled = shuffle_rows(pmap_pontine_5parcels)
    pmap_lang_shuffled = shuffle_rows(pmap_lang_5parcels)

    pmap_mdtb_flat_shuffled = pmap_mdtb_shuffled.flatten()
    pmap_pontine_flat_shuffled = pmap_pontine_shuffled.flatten()
    pmap_lang_flat_shuffled = pmap_lang_shuffled.flatten()

    # Compute correlations with shuffled data
    shuffled_pairs = [
        ("pmap vs mdtb", pmap_flat, pmap_mdtb_flat_shuffled),
        ("pmap vs pontine", pmap_flat, pmap_pontine_flat_shuffled),
        ("pmap vs lang", pmap_flat, pmap_lang_flat_shuffled),
        ("mdtb vs pontine", pmap_mdtb_flat_shuffled, pmap_pontine_flat_shuffled),
        ("mdtb vs lang", pmap_mdtb_flat_shuffled, pmap_lang_flat_shuffled),
        ("pontine vs lang", pmap_pontine_flat_shuffled, pmap_lang_flat_shuffled),
    ]

    for name, arr1, arr2 in shuffled_pairs:
        corr, p_value = stats.pearsonr(arr1, arr2)
        randomized_results[name].append(corr)  

print("\n=== Original vs. Randomized Correlation Results ===")
for name in original_results.keys():
    orig_corr, orig_p = original_results[name]
    rand_corr_mean = np.mean(randomized_results[name]) #computing mean tells us what the expected correlation would be if the data was random (each random test reveals a possible correlation value on a distribution)
    rand_corr_std = np.std(randomized_results[name])

    t_stat, t_p_value = stats.ttest_1samp(randomized_results[name], orig_corr)
    
    print(f"{name}:")
    print(f"  Original Corr: {orig_corr:.4f}")
    print(f"  Randomized Corr (mean ± std): {rand_corr_mean:.4f} ± {rand_corr_std:.4f}")
    print(f"  T-statistic: {t_stat:.4f}, p-value: {t_p_value:.4f}")  # Added t-test results
    print("-" * 50)



=== Original vs. Randomized Correlation Results ===
pmap vs mdtb:
  Original Corr: 0.6917
  Randomized Corr (mean ± std): 0.0393 ± 0.3033
  T-statistic: -15.0591, p-value: 0.0000
--------------------------------------------------
pmap vs pontine:
  Original Corr: 0.6621
  Randomized Corr (mean ± std): 0.0212 ± 0.2980
  T-statistic: -15.0535, p-value: 0.0000
--------------------------------------------------
pmap vs lang:
  Original Corr: 0.5885
  Randomized Corr (mean ± std): -0.0069 ± 0.2763
  T-statistic: -15.0831, p-value: 0.0000
--------------------------------------------------
mdtb vs pontine:
  Original Corr: 0.2430
  Randomized Corr (mean ± std): 0.0534 ± 0.1903
  T-statistic: -6.9768, p-value: 0.0000
--------------------------------------------------
mdtb vs lang:
  Original Corr: 0.2985
  Randomized Corr (mean ± std): 0.0181 ± 0.1936
  T-statistic: -10.1394, p-value: 0.0000
--------------------------------------------------
pontine vs lang:
  Original Corr: 0.1875
  Randomiz